# Join Values from Upper-Level Aggregates to Lower-Level Aggregates - 2010 PA Household Income

## Import Libraries

In [1]:
# Import libraries
import arcpy
import glob
import pandas as pd
import numpy as np
import warnings
from functools import reduce
np.set_printoptions(threshold = sys.maxsize) # view entire NumPy array
warnings.filterwarnings('ignore') # supress warnings

## Read Data

In [2]:
# Read in county and tract-level income data
(county, tract, block_group) = (pd.read_csv(r'data/pa_county_original.csv').drop([0, 0], axis = 0),
                                pd.read_csv(r'data/pa_tract_original.csv').drop([0, 0], axis = 0), 
                                pd.read_csv(r'data/pa_block_group_original.csv'))

## Clean Data

### County-Level Data

In [3]:
# Check for NaN values in the `HC01_EST_VC13` column
county['HC01_EST_VC13'].isna().sum()

0

In [4]:
# Clean the `county` data frame
county_cleaned = pd.DataFrame() 
county_cleaned['county_fips'] = county['GEO.id2']
county_cleaned['county'] = county['GEO.display-label'].str.split(',').str[0]
county_cleaned['county_income'] = county['HC01_EST_VC13']
county_cleaned.head(3)

,county_fips,county,county_income
1,42001,Adams County,56529
2,42003,Allegheny County,47961
3,42005,Armstrong County,42752


In [5]:
# Write 'county_cleaned' data frame to CSV
county_cleaned.to_csv(r'data/2020.02.25_pa_county.csv')

### Tract-Level Data

In [6]:
# Check for NaN values in the `HC01_EST_VC13` column
tract['HC01_EST_VC13'] = tract['HC01_EST_VC13'].replace('-', np.nan)
tract['HC01_EST_VC13'] = tract['HC01_EST_VC13'].replace('250,000+', '250000')
tract['HC01_EST_VC13'].isna().sum()

33

In [7]:
# Get the mean value of the `HC01_EST_VC13` column
tract['HC01_EST_VC13'] = tract['HC01_EST_VC13'].astype(float)
tract['HC01_EST_VC13'].mean()

52834.855886970174

In [8]:
# Clean the `tract` data frame
tract_cleaned = pd.DataFrame() 
tract_cleaned['tract_fips'] = tract['GEO.id2']
tract_cleaned['tract'] = tract['GEO.display-label'].str.split(',').str[0]
tract_cleaned['tract_income'] = tract['HC01_EST_VC13']
tract_cleaned['tract_income'] = tract_cleaned['tract_income'].replace(np.nan, 52835)
tract_cleaned.head(3)

,tract_fips,tract,tract_income
1,42001030101,Census Tract 301.01,70729.0
2,42001030102,Census Tract 301.02,62372.0
3,42001030200,Census Tract 302,52424.0


In [9]:
# Write 'tract_cleaned' data frame to CSV
tract_cleaned.to_csv(r'data/2020.02.25_pa_tract.csv')

### Block Group-Level Data

In [10]:
# Check for NaN values in the `B19013e1` column
block_group['B19013e1'].isna().sum()

49

In [11]:
# Get the mean value of the `B19013e1` column
block_group['B19013e1'].mean()

52552.59735837375

In [12]:
# Clean the `block_group` data frame
block_group_cleaned = pd.DataFrame() 
block_group_cleaned['block_group_fips'] = block_group['GEOID10']
block_group_cleaned['block_group'] = block_group['NAMELSAD10']
block_group_cleaned['block_group_income'] = block_group['B19013e1']
block_group_cleaned['block_group_income'] = block_group_cleaned['block_group_income'].replace(np.nan, 52554)
block_group_cleaned.head(3)

,block_group_fips,block_group,block_group_income
0,420035003001,Block Group 1,38191.0
1,420034994002,Block Group 2,23750.0
2,420034994001,Block Group 1,33750.0


In [13]:
# Write 'block_group_cleaned' data frame to CSV
block_group_cleaned.to_csv(r'data/2020.02.25_pa_block_group.csv')

## Process Data

### Join Values from Upper-Level Aggregates to Lower-Level Aggregates

#### County-Tract Merge

In [14]:
# Merge `tract_cleaned` and `county_cleaned`
county_tract = tract_cleaned.merge(county_cleaned, 
                                   left_on = tract_cleaned['tract_fips'].astype('str').str[0:5],
                                   right_on = county_cleaned['county_fips'].astype('str'))
county_tract = county_tract.drop(['key_0'], axis = 1)

In [15]:
# Add `state` and `state_fips` columns
county_tract['state'] = 'Pennsylvania'
county_tract['state_fips'] = '42'

In [16]:
# Reorder columns
county_tract = county_tract[['state', 'state_fips', 'county', 'county_fips', 'county_income', 'tract', 'tract_fips', 
                             'tract_income']]
county_tract.head(3)

,state,state_fips,county,county_fips,county_income,tract,tract_fips,tract_income
0,Pennsylvania,42,Adams County,42001,56529,Census Tract 301.01,42001030101,70729.0
1,Pennsylvania,42,Adams County,42001,56529,Census Tract 301.02,42001030102,62372.0
2,Pennsylvania,42,Adams County,42001,56529,Census Tract 302,42001030200,52424.0


In [17]:
# Write `county_tract` to CSV
county_tract.to_csv(r'data/2020.02.25_pa_county_tract.csv')

#### County-Block Group Merge

In [18]:
# Merge `block_group_cleaned` and `county_cleaned`
county_block_group = block_group_cleaned.merge(county_cleaned, 
                                               left_on = block_group_cleaned['block_group_fips'].astype('str').str[:5], 
                                               right_on = county_cleaned['county_fips'].astype('str'))
county_block_group = county_block_group.drop(['key_0'], axis = 1)

In [19]:
# Add `state` and `state_fips` columns
county_block_group['state'] = 'Pennsylvania'
county_block_group['state_fips'] = '42'

In [20]:
# Reorder columns
county_block_group = county_block_group[['state', 'state_fips', 'county', 'county_fips', 'county_income', 'block_group', 
                                         'block_group_fips', 'block_group_income']]
county_block_group.head(3)

,state,state_fips,county,county_fips,county_income,block_group,block_group_fips,block_group_income
0,Pennsylvania,42,Allegheny County,42003,47961,Block Group 1,420035003001,38191.0
1,Pennsylvania,42,Allegheny County,42003,47961,Block Group 2,420034994002,23750.0
2,Pennsylvania,42,Allegheny County,42003,47961,Block Group 1,420034994001,33750.0


In [21]:
# Write `county_block_group` to CSV
county_block_group.to_csv(r'data/2020.02.25_pa_county_block_group.csv')

#### Tract-Block Merge

In [22]:
# Merge `block_group_cleaned` and `tract_cleaned`
tract_block_group = block_group_cleaned.merge(tract_cleaned, 
                                              left_on = block_group_cleaned['block_group_fips'].astype('str').str[:-1], 
                                              right_on = tract_cleaned['tract_fips'].astype('str'))
tract_block_group = tract_block_group.drop(['key_0'], axis = 1)

In [23]:
# Add `state` and `state_fips` columns
tract_block_group['state'] = 'Pennsylvania'
tract_block_group['state_fips'] = '42'
tract_block_group.head(3)

,block_group_fips,block_group,block_group_income,tract_fips,tract,tract_income,state,state_fips
0,420035003001,Block Group 1,38191.0,42003500300,Census Tract 5003,36719.0,Pennsylvania,42
1,420035003004,Block Group 4,24804.0,42003500300,Census Tract 5003,36719.0,Pennsylvania,42
2,420035003003,Block Group 3,39408.0,42003500300,Census Tract 5003,36719.0,Pennsylvania,42


In [24]:
# Reorder columns
tract_block_group = tract_block_group[['state', 'state_fips', 'tract', 'tract_fips', 'tract_income', 'block_group', 
                                         'block_group_fips', 'block_group_income']]
tract_block_group.head(3)

,state,state_fips,tract,tract_fips,tract_income,block_group,block_group_fips,block_group_income
0,Pennsylvania,42,Census Tract 5003,42003500300,36719.0,Block Group 1,420035003001,38191.0
1,Pennsylvania,42,Census Tract 5003,42003500300,36719.0,Block Group 4,420035003004,24804.0
2,Pennsylvania,42,Census Tract 5003,42003500300,36719.0,Block Group 3,420035003003,39408.0


In [25]:
# Write `tract_block_group` to CSV
county_tract.to_csv(r'data/2020.02.25_pa_tract_block_group.csv')

#### Merge All Three Levels of Aggregation

In [26]:
# Convert columns types
block_group_cleaned['block_group_fips'] = block_group_cleaned['block_group_fips'].astype(np.str)
tract_cleaned['tract_fips'] = tract_cleaned['tract_fips'].astype(np.str)
county_cleaned['county_fips'] = county_cleaned['county_fips'].astype(np.str)

In [27]:
# Add `tract_fips` and `county_fips` columns to `block_group_cleaned
block_group_cleaned['tract_fips'] = block_group_cleaned['block_group_fips'].str[:-1]
block_group_cleaned['county_fips'] = block_group_cleaned['block_group_fips'].str[:-7]

In [28]:
# Merge all three levels of aggregation
data = block_group_cleaned.merge(tract_cleaned, how = 'left', on = 'tract_fips')
data = data.merge(county_cleaned, how = 'left', on ='county_fips')

In [29]:
# Add `state` and `state_fips` columns
data['state'] = 'Pennsylvania'
data['state_fips'] = '42'

In [30]:
# Reorder columns
data = data[['state', 'state_fips', 'county', 'county_fips', 'county_income', 'tract', 'tract_fips', 'tract_income', 
             'block_group', 'block_group_fips', 'block_group_income']]
data.head(3)

,state,state_fips,county,county_fips,county_income,tract,tract_fips,tract_income,block_group,block_group_fips,block_group_income
0,Pennsylvania,42,Allegheny County,42003,47961,Census Tract 5003,42003500300,36719.0,Block Group 1,420035003001,38191.0
1,Pennsylvania,42,Allegheny County,42003,47961,Census Tract 4994,42003499400,29391.0,Block Group 2,420034994002,23750.0
2,Pennsylvania,42,Allegheny County,42003,47961,Census Tract 4994,42003499400,29391.0,Block Group 1,420034994001,33750.0


In [31]:
# Write `data` to CSV
data.to_csv(r'data/2020.02.25_pa_data.csv')

### Reproduce Table 1 (p. 41)

In [32]:
# Create a data frame for Table 1
table1 = pd.DataFrame() 

In [33]:
# Add rows to `table1` 
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.01' and " + 
                                  "block_group == 'Block Group 1'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.03' and " +  
                                  "block_group == 'Block Group 1'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.03' and " +  
                                  "block_group == 'Block Group 2'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.03' and " +  
                                  "block_group == 'Block Group 3'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.03' and " +  
                                  "block_group == 'Block Group 4'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.04' and " +  
                                  "block_group == 'Block Group 1'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.04' and " +  
                                  "block_group == 'Block Group 2'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 8.04' and " +  
                                  "block_group == 'Block Group 3'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 9.01' and " +  
                                  "block_group == 'Block Group 1'"))
table1 = table1.append(data.query("county == 'Philadelphia County' and tract == 'Census Tract 9.01' and " +  
                                  "block_group == 'Block Group 2'"))

In [34]:
# Drop columns
table1 = table1.drop(['state_fips', 'county_fips', 'tract_fips', 'block_group_fips'], axis = 1)
table1

,state,county,county_income,tract,tract_income,block_group,block_group_income
8223,Pennsylvania,Philadelphia County,36251,Census Tract 8.01,79000.0,Block Group 1,79000.0
8825,Pennsylvania,Philadelphia County,36251,Census Tract 8.03,59135.0,Block Group 1,40795.0
7608,Pennsylvania,Philadelphia County,36251,Census Tract 8.03,59135.0,Block Group 2,51574.0
8827,Pennsylvania,Philadelphia County,36251,Census Tract 8.03,59135.0,Block Group 3,106658.0
7605,Pennsylvania,Philadelphia County,36251,Census Tract 8.03,59135.0,Block Group 4,107727.0
7841,Pennsylvania,Philadelphia County,36251,Census Tract 8.04,62589.0,Block Group 1,64643.0
7607,Pennsylvania,Philadelphia County,36251,Census Tract 8.04,62589.0,Block Group 2,72431.0
7604,Pennsylvania,Philadelphia County,36251,Census Tract 8.04,62589.0,Block Group 3,48750.0
7845,Pennsylvania,Philadelphia County,36251,Census Tract 9.01,39265.0,Block Group 1,53897.0
7844,Pennsylvania,Philadelphia County,36251,Census Tract 9.01,39265.0,Block Group 2,25810.0
